In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, uniform
from kan import *
import torch
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams.update({
    'font.size': 12,               # Font size
    'axes.labelsize': 15,          # Label size
    'axes.titlesize': 15,          # Title size
    'axes.linewidth': 1.2,         # Axis line width
    'xtick.labelsize': 10,         # Size of x-axis tick labels
    'ytick.labelsize': 10,         # Size of y-axis tick labels
    'xtick.major.size': 6,         # Length of major ticks on the x-axis
    'ytick.major.size': 6,         # Length of major ticks on the y-axis
    'xtick.major.width': 1.0,      # Width of major ticks on the x-axis
    'ytick.major.width': 1.0,      # Width of major ticks on the y-axis
    'figure.dpi': 300,             # Image resolution
    'savefig.dpi': 300,            # Resolution for saving figures
    'figure.figsize': (12, 5),     # Figure size
})

## Scenario C

#### $(I_{\mathbf{x}}| Y_{\mathbf{x}} > u) \sim \mathrm{Multi}_3(m_{\text{C1}}(\mathbf{x}; u),m_{\text{C2}}(\mathbf{x}; u),m_{\text{C3}}(\mathbf{x}; u))$ given $\mathbf{X}=\mathbf{x}$

In [ ]:
def MC_C(M, n):
    def m1(x1, x2):
        mean = [np.exp(-u), np.exp(-u)] 
        cov = [[1, 0], [0, 1]]
        return multivariate_normal.cdf([x1, x2], mean=mean, cov=cov)

    def m2(x1, x2):
        return 0.5 + 0.4 *np.exp(- x1) * np.cos(2 * np.pi * x2) + 1/u**2

    def m3(x1, x2):
        return 0.8 * x2 * np.sin(np.pi * x1)**2 + 1/(u**2)
    
    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 

    for i in range(M):
        np.random.seed(i)
        
        # X1 y X2 ~ U(0,1)
        X1 = uniform.rvs(loc=0, scale=1, size=n)
        X2 = uniform.rvs(loc=0, scale=1, size=n)

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u = np.quantile(Y_X, u_quantile)
        mask = Y_X > u

        probs = np.zeros((n, 3))
        probs[:, 0] = np.array([m1(x1, x2) for x1, x2 in zip(X1, X2)])
        probs[:, 1] = np.array([m2(x1, x2) for x1, x2 in zip(X1, X2)])
        probs[:, 2] = np.array([m3(x1, x2) for x1, x2 in zip(X1, X2)])

        probs_sum = probs.sum(axis=1)
        probs = probs / probs_sum[:, np.newaxis]

        I = np.zeros(n, dtype=int)
        I[mask] = [np.random.multinomial(1, pvals).argmax() for pvals in probs[mask]]

        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        x_train_tensor = torch.tensor(df_sim[['X1', 'X2']].values, dtype=torch.float32)
        y_train_tensor = torch.tensor(df_sim['I'].values, dtype=torch.long)

        model = KAN(width=[2, 3], grid=2, k=3)
        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)
        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z.reshape(100, 100, 3)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all, xx, yy = MC_C(500, 15000)

In [ ]:
def m1(x1, x2):
    return multivariate_normal.cdf([x1, x2], mean=[0,0], cov=[[1, 0], [0, 1]])

def m2(x1, x2):
    return 0.5 + 0.4 * np.exp(- x1) * np.cos(2 * np.pi * x2)

def m3(x1, x2):
    return 0.8 * x2 * np.sin(np.pi * x1)**2

probabilities_all = np.array(probabilities_all)
monte_carlo_mean_surface = np.mean(probabilities_all, axis=0)

f_values = np.zeros((10000, 3)) 
f_values[:, 0] = np.array([m1(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])
f_values[:, 1] = np.array([m2(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])
f_values[:, 2] = np.array([m3(x1, x2) for x1, x2 in zip(xx.ravel(), yy.ravel())])

f_values_sum = f_values.sum(axis=1)
f_values = f_values / f_values_sum[:, np.newaxis]

f_values = f_values.reshape(100, 100, 3)

squared_difference_1 = (monte_carlo_mean_surface[:, :, 0] - f_values[:, :, 0]) ** 2
squared_difference_2 = (monte_carlo_mean_surface[:, :, 1] - f_values[:, :, 1]) ** 2
squared_difference_3 = (monte_carlo_mean_surface[:, :, 2] - f_values[:, :, 2]) ** 2

integral_1 = np.trapz(squared_difference_1, x=yy[:, 0], axis=0)
integral_2 = np.trapz(squared_difference_2, x=yy[:, 0], axis=0)
integral_3 = np.trapz(squared_difference_3, x=yy[:, 0], axis=0)

mise_1 = np.trapz(integral_1, x=xx[0, :])
mise_2 = np.trapz(integral_2, x=xx[0, :])
mise_3 = np.trapz(integral_3, x=xx[0, :])

print('MISE C1: ' + str(mise_1))
print('MISE C2: ' + str(mise_2))
print('MISE C3: ' + str(mise_3))

fig_labels = ['1', '2', '3']
for i in range(3):
    fig = plt.figure(figsize=(12, 5))
    gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)
    
    # Estimated POC Surface
    ax1 = fig.add_subplot(gs[0])
    im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface[:, :, i], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
    ax1.set_xlabel(r'$x_1$', fontsize=16)
    ax1.set_ylabel(r'$x_2$', fontsize=16)
    ax1.set_title(r'MC Mean KANE POC Surface $j = ' + f'{fig_labels[i]}' + r'$')
    
    # True POC Surface
    ax2 = fig.add_subplot(gs[1])
    im2 = ax2.pcolor(xx, yy, f_values[:, :, i], shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
    ax2.set_xlabel(r'$x_1$', fontsize=16)
    ax2.set_ylabel(r'$x_2$', fontsize=16)
    ax2.set_title(r'True POC Surface $j = ' + f'{fig_labels[i]}' + r'$')

    cbar_ax = fig.add_subplot(gs[2])
    fig.colorbar(im2, cax=cbar_ax)

    pos = cbar_ax.get_position() 
    cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 
    
    if i == 0:
        plt.suptitle('Scenario C (n = 15 000)', fontsize=18, fontweight='bold')

    plt.tight_layout()
    plt.show()